In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [20]:
from housepricemanual import *

In [ ]:
# !type housepricemanual.py

In [ ]:
# !dir PATH

## Get data

In [3]:
PATH = Path('../data/houseprice/')

In [4]:
x_train, y_train, x_valid, y_valid = get_data()

In [5]:
n, m = x_train.shape
# c = y_train.max()+1
# x_train, x_train.shape, y_train, y_train.shape, y_train.min(), y_train.max()

In [6]:
train_mean, train_std = x_train.mean(), x_train.std()
train_mean, train_std

(tensor(0.1351), tensor(0.3354))

In [7]:
x_train = normalize(x_train, train_mean, train_std)
x_valid = normalize(x_valid, train_mean, train_std)

### Check mean, std

In [ ]:
x_train.mean(), x_train.std()

In [ ]:
y_train.shape

In [ ]:
assert n==y_train.shape[0]==1168

## Initial python model

### Matmul

In [ ]:
x_train.shape, x_valid.shape

In [ ]:
weights = torch.randn(481, 10)
bias = torch.zeros(10)

In [ ]:
weights, weights.shape, bias

In [ ]:
def matmul(a,b):
    ar,ac = a.shape
    br,bc = b.shape
    assert ac==br
    c = torch.zeros(ar,bc)
    for i in range(ar):
        for j in range(bc):
            for k in range(ac):
                c[i,j] += a[i,k] * b[k,j]
    return c

In [ ]:
m1 = x_valid[:5]
m2 = weights

In [ ]:
m1.shape, m2.shape

In [ ]:
%time t1 = matmul(m1,m2)

In [ ]:
t1.shape

In [ ]:
len(x_train)

In [ ]:
def matmul(a,b):
    ar,ac = a.shape
    br,bc = b.shape
    assert ac==br
    c = torch.zeros(ar,bc)
    
    for i in range(ar):
        for j in range(bc):
            c[i,j] = (a[i,:] * b[:,j]).sum()
    return c

In [ ]:
%timeit -n 10 _=matmul(m1, m2)

In [ ]:
t1.dtype

In [ ]:
def near(a,b): return torch.allclose(a, b, rtol=1e-3, atol=1e-5)
def test_near(a,b): test(a,b,near)

In [ ]:
test_near(t1, matmul(m1, m2))

#### Broadcasting

In [ ]:
mm = tensor([[1., 2, 3], [4,5,6], [7,8,9]]); mm

In [ ]:
cc = tensor([10., 20, 30]); cc

In [ ]:
2*mm

In [ ]:
tt = cc.expand_as(mm); tt

In [ ]:
mm+tt

In [ ]:
tt.storage()

In [ ]:
cc, cc.shape, cc.unsqueeze(0)

#### Matmul with broadcasting

In [ ]:
def matmul(a,b):
    ar,ac = a.shape
    br,bc = b.shape
    assert ac==br
    c = torch.zeros(ar,bc)
    for i in range(ar):
        c[i] = (a[i].unsqueeze(-1) * b).sum(dim=0)
    return c

In [ ]:
%timeit -n 10 _=matmul(m1,m2)

In [ ]:
test_near(t1, matmul(m1,m2))

#### Matmul with Pytorch op

In [ ]:
%timeit -n 10 t2 = m1.matmul(m2)

In [ ]:
t2 = m1@m2

In [ ]:
test_near(t1, t2)

In [ ]:
m1.shape, m2.shape

### Basic architecture - weight init

#### Manual kaiming init

In [ ]:
# num hidden
nh = 50

In [ ]:
n, m

In [ ]:
# simplified kaiming init
w1 = torch.randn(m, nh)/math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh, 1)/math.sqrt(nh)
b2 = torch.zeros(1)

In [ ]:
w1.mean(), w2.std()

In [ ]:
test_near_zero(w1.mean())
test_near_zero(1/math.sqrt(m)-w1.std())

In [ ]:
w1.mean(), w1.std()*math.sqrt(m)

In [ ]:
# Approx. ~(0,1) because of the normalization with training mean and std
x_valid.mean(), x_valid.std()

In [ ]:
def lin(x, w, b):
    return x@w+b

In [ ]:
t = lin(x_train, w1, b1)

In [ ]:
# This should also be ~(0,1) because of kaiming init
t.mean(), t.std()

In [ ]:
def relu(x):
    return x.clamp_min(0.)

In [ ]:
t = relu(lin(x_train, w1, b1))

In [ ]:
# This is not ~(0,1) because of the ReLU function
t.mean(), t.std()

In [ ]:
# Kaiming init for ReLU
w1 = torch.randn(m, nh) * math.sqrt(2/m)

In [ ]:
w1.mean(), w1.std(), w1.std()/math.sqrt(2/m)

In [ ]:
t = relu(lin(x_train, w1, b1))
t.mean(), t.std()

#### PyTorch kaiming for w1

In [ ]:
# num hidden
nh = 50

In [ ]:
# Torch kaiming init
w1 = torch.zeros(m, nh)
w1 = init.kaiming_normal_(w1, mode='fan_out')
t = relu(lin(x_train, w1, b1))

In [ ]:
t.mean(), t.std()

In [ ]:
init.kaiming_normal_??

In [ ]:
import torch.nn

In [ ]:
torch.nn.Linear(m, nh).weight.shape

In [ ]:
# Adjusted ReLU
def relu(x):
    return x.clamp_min(0.)-0.5

In [ ]:
# Using kaiming init and ReLU with adjusted mean
w1 = torch.zeros(m, nh)
init.kaiming_normal_(w1, mode='fan_out')
t = relu(lin(x_train, w1, b1))
t.mean(), t.std()

#### Full kaiming init pytorch

In [8]:
# num hidden
nh = 50

In [9]:
w1 = torch.zeros(m, nh)
init.kaiming_normal_(w1, mode='fan_out')
b1 = torch.zeros(nh)
w2 = torch.zeros(nh, 1)
init.kaiming_normal_(w2, mode='fan_out')
b2 = torch.zeros(1)

#### Simple model

In [ ]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [ ]:
%timeit -n 10 _=model(x_valid)

In [ ]:
assert model(x_valid).shape==torch.Size([x_valid.shape[0], 1])

### Loss function: MSE

In [ ]:
model(x_valid).shape

In [ ]:
mse(model(x_valid), y_valid)

### Loss function: RMSLE

In [10]:
def rmsle(x, y):
    """
    Compute the Root Mean Squared Log Error for hypthesis h and targets y
    
    Args:
        h - numpy array containing predictions with shape (n_samples, n_targets)
        y - numpy array containing targets with shape (n_samples, n_targets)
    """
    return torch.sqrt( ((torch.log(x + 1) - torch.log(y + 1))**2).mean() )

Same as

``nn.functional.mse_loss((model(xb)+1).log().squeeze(), (yb+1).log()).sqrt()``

### Gradients and backward pass

In [ ]:
#gradient of the loss function wrt the output of the previous layer (lin with w2, b2)
def mse_grad(inp, targ):
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]

In [ ]:
def lin_grad(inp, out, w, b):
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [ ]:
def relu_grad(inp, out):
    # grad of relu wrt input
    inp.g = (inp>0).float() * out.g

In [ ]:
def forward_and_backward(inp, targ):
    # forward
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    loss = mse(out, targ)
    
    # backward
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

In [ ]:
forward_and_backward(x_train, y_train)

In [ ]:
x_train.g

In [ ]:
# Save for testing against later
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig  = x_train.g.clone()

Use ``torch.autograd`` to check the results

In [ ]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [ ]:
def forward(inp, targ):
    # forward
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    return mse(out, targ)

In [ ]:
loss = forward(xt2, y_train)

In [ ]:
loss.backward()

In [ ]:
test_near(w22.grad, w2.g)
test_near(b22.grad, b2.g)
test_near(w12.grad, w1.g)
test_near(b12.grad, b1.g)
test_near(xt2.grad, x_train.g)

## Refactor model

### Layers as classes

#### No superclass

In [ ]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)-0.5
        return self.out
    
    def backward(self):
        self.inp.g = (self.inp>0).float() * self.out.g

In [ ]:
class Lin():
    def __init__(self, w, b):
        self.w = w
        self.b = b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = self.inp @ self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum()       

In [ ]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp - targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2.*(self.inp.squeeze()-self.targ).unsqueeze(-1) / self.inp.shape[0]

In [ ]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1, b1), Relu(), Lin(w2, b2)]
        self.loss = Mse()
    
    def __call__(self, x, targ):
        for l in self.layers:
            x = l(x)
        return self.loss(x, targ)
        # the x in self.loss(x, targ) is the output of the last layer (Lin(w2,b2)) and gets its gradient defined here.
        # This is why out.g is available when called in backward for the first of the reversed layers.

    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers):
            l.backward()

In [ ]:
w1.g, b1.g, w2.g, b2.g = [None]*4

In [ ]:
model = Model(w1, b1, w2, b2)

In [ ]:
%time loss = model(x_valid, y_valid)

In [ ]:
%time loss = model(x_train, y_train)

In [ ]:
%time model.backward()

#### Module superclass with Module.forward()

In [ ]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*self.args)
        return self.out
    
    def forward(self):
        raise Exception(f"not implemented")
    
    def backward(self):
        return self.bwd(self.out, *self.args)

In [ ]:
class Relu(Module):
    def forward(self, inp):
        return inp.clamp_min(0.) - 0.5
    
    def bwd(self, out, inp):
        inp.g = (inp>0).float() * out.g

In [ ]:
class Lin(Module):
    def __init__(self, w, b):
        self.w, self.b = w, b

    def forward(self, inp):
        return inp @ self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = torch.einsum("bi, bj -> ij", inp, out.g)
        self.b.g = out.g.sum(0)

In [ ]:
class Mse(Module):
    def forward(self, inp, targ):
        return (inp.squeeze() - targ).pow(2).mean()
    
    def bwd(self, out, inp, targ):
        inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]

In [ ]:
class Model():
    def __init__(self):
        self.layers = [Lin(w1, b2), Relu(), Lin(w2, b2)]
        self.loss = Mse()
    
    def __call__(self, x, targ):
        for l in self.layers:
            x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers):
            l.backward()

In [ ]:
w1.g, b1.g, w2.g, b2.g = [None]*4
model = Model()

In [ ]:
%time loss = model(x_train, y_train)

In [ ]:
%time model.backward()

In [ ]:
x_train.shape, w1.shape, w2.shape

#### Class Lin() without torch.einsum

In [ ]:
class Lin(Module):
    def __init__(self, w, b):
        self.w, self.b = w, b
        
    def forward(self, inp):
        return inp @ self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = inp.t() @ out.g
        self.b.g = out.g.sum(0)

In [ ]:
model = Model()

In [ ]:
%time loss = model(x_train, y_train)

In [ ]:
%time model.backward()

#### nn.Linear and nn.Module

In [ ]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in, nh), nn.ReLU(), nn.Linear(nh, n_out)]
        self.loss = mse
    
    def __call__(self, x, targ):
        for l in self.layers:
            x = l(x)
        return self.loss(x.squeeze(), targ)

In [ ]:
model = Model(m, nh, 1)

In [ ]:
%time loss = model(x_train, y_train)

In [ ]:
%time loss.backward()

### Training loop

In [11]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in, nh), nn.ReLU(), nn.Linear(nh, n_out)]
        self.loss = mse
    
    def __call__(self, x):
        for l in self.layers:
            x = l(x)
        return x

In [12]:
bs = 64               # batchsize
lr = 0.5              # learning rate
epochs = 1
xb, yb = x_train[:bs], y_train[:bs]

In [13]:
loss_func = rmsle

In [14]:
loss_func = nn.MSELoss()

#### fit()

In [15]:
model = Model(m, nh, 1)

In [16]:
def fit():
    for e in range(epochs):
        for i in range((n-1)//bs + 1):
            start_i = i * bs
            end_i = start_i + bs
            xb, yb = x_train[start_i:end_i], y_train[start_i:end_i]
            loss = loss_func(model(xb).squeeze(), yb)
            
            loss.backward()
            with torch.no_grad():
                for p in model.parameters():
                    p -= p.grad * lr
                model.zero_grad()

In [17]:
fit()

#### Simplify model to use nn.Sequential

In [18]:
model = nn.Sequential(nn.Linear(m, nh), nn.ReLU(), nn.Linear(nh, 1))

In [19]:
fit()

### optim

In [21]:
opt = optim.SGD(model.parameters(), lr=lr)